This code block handles the import statements and data loading, to keep the notebook clean.

In [2]:
import pandas as pd
import kagglehub

#Philadelphia 2025 Crime Statistics, from our github
url = "data/philadelphia_crime_data_2025.csv"
phil_test = pd.read_csv(url)

#this downloads the kaggle nfl dataset
path_nfl = kagglehub.dataset_download("keonim/nfl-game-scores-dataset-2017-2023")
game_dates = pd.read_csv(f"{path_nfl}/Season_Scores/2024_scores.csv")


In [3]:
phil_test

,the_geom,cartodb_id,the_geom_webmercator,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng
0,0101000020E610000078D60B9155CC52C00609171317FB...,1,0101000020110F0000365F9FE53BEE5FC11CBBED6D748A...,27244917,16,1,2025-02-27 01:16:00+00,2025-02-26,20:16:00,20,2.025160e+11,400 BLOCK N 35TH ST,300,Robbery No Firearm,-75.192723,39.961642,39.961642,-75.192723
1,0101000020E6100000205EFA3EECC952C0C8AFBCA424FD...,2,0101000020110F000075DADA5023EA5FC1F87526DCBA8C...,27742666,22,3,2025-03-23 21:15:00+00,2025-03-23,17:15:00,17,2.025220e+11,1200 BLOCK CECIL B MOORE AV,400,Aggravated Assault No Firearm,-75.155044,39.977681,39.977681,-75.155044
2,0101000020E6100000C8BB2A1DDACA52C06845954DB8F9...,3,0101000020110F00004F32C35BB7EB5FC19CDDA2C8EF88...,27742672,9,3,2025-03-23 20:19:00+00,2025-03-23,16:19:00,16,2.025090e+11,100 BLOCK S 17TH ST,400,Aggravated Assault No Firearm,-75.169563,39.950937,39.950937,-75.169563
3,0101000020E610000080102393B2C152C020440F42E10C...,4,0101000020110F000005140F9F2ADC5FC183209DE12F9E...,27742741,7,2,2025-03-23 22:12:00+00,2025-03-23,18:12:00,18,2.025070e+11,900 BLOCK RED LION RD,400,Aggravated Assault No Firearm,-75.026524,40.100624,40.100624,-75.026524
4,0101000020E6100000B818A83186CB52C00C1F32618BFD...,5,0101000020110F0000D26359A7DBEC5FC168F628BA2C8D...,27245131,22,4,2025-02-27 13:27:00+00,2025-02-27,08:27:00,8,2.025220e+11,1600 BLOCK N 28TH ST,400,Aggravated Assault Firearm,-75.180066,39.980816,39.980816,-75.180066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55997,0101000020E61000006B892A3685C952C02ABCF4C42DFF...,3392169,0101000020110F0000BF8B2F4D74E95FC104097180FC8E...,24722516,25,4,2025-01-09 18:38:00+00,2025-01-09,13:38:00,13,2.025250e+11,1000 BLOCK W LEHIGH AV,600,Thefts,-75.148756,39.993584,39.993584,-75.148756
55998,0101000020E61000004CC3F01131C052C07520EBA9D505...,3392177,0101000020110F0000E61787CD9BD95FC103E9531F5E96...,25223158,8,1,2025-01-28 18:11:00+00,2025-01-28,13:11:00,13,2.025080e+11,4700 BLOCK Tolbut St,600,Theft from Vehicle,-75.002995,40.045583,40.045583,-75.002995
55999,0101000020E6100000442E7A9F91C952C0125464DFDBF4...,3392182,0101000020110F0000801A2B6289E95FC191AD4A7A8D83...,24169443,3,2,2025-01-01 21:27:00+00,2025-01-01,16:27:00,16,2.025030e+11,2700 BLOCK S FRONT ST,600,Thefts,-75.149513,39.912960,39.912960,-75.149513
56000,0101000020E610000091A3EF2498CF52C0D481ACA7D6F5...,3392189,0101000020110F00002460DB82C5F35FC128F27A2DA384...,24628289,12,2,2025-01-19 07:07:00+00,2025-01-19,02:07:00,2,2.025120e+11,7000 BLOCK GREENWAY AV,400,Aggravated Assault No Firearm,-75.243661,39.920613,39.920613,-75.243661


In [56]:
def nfl_game_date(year, team):
    # load the specific year
    df = pd.read_csv(f"{path_nfl}/Season_Scores/{year}_scores.csv")
    df = df.dropna(subset=['Date'])

    # subsets the df to games where the team is playing
    df_away = df[(df["AwayTeam"] == team)]
    df_home = df[(df["HomeTeam"] == team)]

    home_dates_with_year = df_home['Date'].apply(
        lambda x: f"{x}/{year+1}" if int(str(x).split('/')[0]) <= 6 
        else f"{x}/{year}")    
    away_dates_with_year = df_away['Date'].apply(
        lambda x: f"{x}/{year+1}" if int(str(x).split('/')[0]) <= 6 
        else f"{x}/{year}")

    # Now convert to datetime
    home_game_dates = pd.to_datetime(home_dates_with_year)
    away_game_dates = pd.to_datetime(away_dates_with_year)

    # Gets the result of the game
    home_game_win = df_home['HomeWin'].to_list()
    away_game_win = df_away['AwayWin'].to_list()

    # Convert to "Win" or "Loss"
    home_game_win = ["Win" if bool(x) else "Loss" for x in home_game_win]
    away_game_win = ["Win" if bool(x) else "Loss" for x in away_game_win]

    # Creates an away, home df, with the pairs for results and tags the Location
    home = pd.DataFrame(list(zip(home_game_dates, home_game_win)))
    away = pd.DataFrame(list(zip(away_game_dates, away_game_win)))
    home["Location"] = "Home"
    away["Location"] = "Away"

    # Puts the two together and sorts them by date
    season = pd.concat([home, away], ignore_index=True) 
    season.columns = ["Date", "Result", "Location"]
    season = season.sort_values('Date').reset_index(drop=True)    

    return season


In [5]:
nfl_game_date(2024, "Eagles")

,Date,Result,Location
0,2024-08-09,True,Away
1,2024-08-15,True,Away
2,2024-08-24,False,Home
3,2024-09-06,True,Home
4,2024-09-16,False,Home
5,2024-09-22,True,Away
6,2024-09-29,False,Away
7,2024-10-13,True,Home
8,2024-10-20,True,Away
9,2024-10-27,True,Away


In [6]:
#this downloads the kaggle nba dataset
path_nba = kagglehub.dataset_download("eoinamoore/historical-nba-data-and-player-box-scores")

100%|██████████████████████████████████████| 93.0M/93.0M [00:03<00:00, 24.8MB/s]

Extracting files...


In [7]:
def nba_game_date(team,year):
    #first get teamId for this dataset
    team_df = pd.read_csv(f"{path_nba}/TeamHistories.csv")

    #searches the dataset for the ID, returns nothing if not found
    team_id = team_df[team_df['teamName'] == team]['teamId']
    if not team_id.empty:
        team_id = team_id.iloc[0]
    else:
        print("Team not found in dataset, look at the documentation.")
        return

    #use team_id and year to sort games wanted
    df = pd.read_csv(f"{path_nba}/Games.csv", low_memory=False)
    df["gameDate"] = pd.to_datetime(df["gameDate"])
    start = pd.to_datetime(f"{year}-10-01")
    end = pd.to_datetime(f"{year+1}-06-30")
    df_home = df[(df["hometeamId"] == team_id) & ((df['gameDate'] >= start) & (df['gameDate'] <= end))]
    df_away = df[(df["awayteamId"] == team_id) & ((df['gameDate'] >= start) & (df['gameDate'] <= end))]


    home_game_dates = df_home["gameDate"].to_list()
    away_game_dates = df_away["gameDate"].to_list()
    
    home_game_win = [x == team_id for x in df_home['winner']]
    away_game_win = [x == team_id for x in df_away['winner']]

    #Creates an away, home df, with the pairs for results and tags the Location
    home = pd.DataFrame(list(zip(home_game_dates, home_game_win)))
    away = pd.DataFrame(list(zip(away_game_dates, away_game_win)))
    home["Location"] = "Home"
    away["Location"] = "Away"
    season = pd.concat([home, away], ignore_index=True) 
    season.columns = ["Date", "Result", "Location"]
    season = season.sort_values('Date').reset_index(drop=True)    
    
    return season

In [8]:
nba_game_date("76ers",2024)

,Date,Result,Location
0,2024-10-11 19:00:00,False,Away
1,2024-10-12 20:00:00,False,Away
2,2024-10-14 19:30:00,True,Away
3,2024-10-16 19:00:00,True,Home
4,2024-10-18 19:00:00,False,Away
...,...,...,...
82,2025-04-05 19:00:00,False,Home
83,2025-04-07 19:30:00,False,Away
84,2025-04-09 19:00:00,True,Away
85,2025-04-11 19:00:00,False,Home


In [9]:
#average number of crimes per day of the week
def average_crimes_per_day_of_week(df, datetime_column):
    df[datetime_column] = pd.to_datetime(df[datetime_column])
    df['date'] = df[datetime_column].dt.date
    df['day_of_week'] = df[datetime_column].dt.day_name()

    daily_counts = df.groupby(['date', 'day_of_week']).size().reset_index(name='daily_crime_count')
    avg_by_day = daily_counts.groupby('day_of_week')['daily_crime_count'].mean().reset_index()
    avg_by_day = avg_by_day.rename(columns={'daily_crime_count': 'avg_crimes_per_day'})
    return avg_by_day

#average number of crimes by type of crime per day
def average_specific_crimes_per_day(df, datetime_column, crime_type_column='text_general_code'):
    df[datetime_column] = pd.to_datetime(df[datetime_column])
    df['date'] = df[datetime_column].dt.date

    daily_counts = df.groupby(['date', crime_type_column]).size().reset_index(name='daily_crime_count')
    avg_by_type = daily_counts.groupby(crime_type_column)['daily_crime_count'].mean().reset_index()
    avg_by_type = avg_by_type.rename(columns={'daily_crime_count': 'avg_crimes_per_day'})
    return avg_by_type

# Philadelphia 2024 baseline
crime_df_philly_2024 = pd.read_csv("data/philadelphia_crime_data_2024.csv")
philly_avg_by_day = average_crimes_per_day_of_week(crime_df_philly_2024, datetime_column='dispatch_date_time')
philly_avg_by_type = average_specific_crimes_per_day(crime_df_philly_2024, datetime_column='dispatch_date_time')

# Chicago 2024–2025
crime_df_chicago = pd.read_csv("data/chicago_crime_data_2024_2025.csv")
chicago_avg_by_day = average_crimes_per_day_of_week(crime_df_chicago, datetime_column='Date')
chicago_avg_by_type = average_specific_crimes_per_day(crime_df_chicago, datetime_column='Date', crime_type_column='Primary Type')

/var/folders/xt/d01l_q856c9g34hfhlqrwxch0000gn/T/ipykernel_21678/1967450083.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[datetime_column] = pd.to_datetime(df[datetime_column])


In [10]:
philly_avg_by_day

,day_of_week,avg_crimes_per_day
0,Friday,442.788462
1,Monday,464.113208
2,Saturday,393.769231
3,Sunday,376.519231
4,Thursday,452.192308
5,Tuesday,471.377358
6,Wednesday,465.634615


In [11]:
philly_avg_by_type.head()

,text_general_code,avg_crimes_per_day
0,Aggravated Assault Firearm,7.698630
1,Aggravated Assault No Firearm,15.281421
2,All Other Offenses,39.051913
3,Arson,2.347403
4,Burglary Non-Residential,4.441011


In [12]:
chicago_avg_by_day

,day_of_week,avg_crimes_per_day
0,Friday,694.569444
1,Monday,686.111111
2,Saturday,694.680556
3,Sunday,665.847222
4,Thursday,665.583333
5,Tuesday,668.055556
6,Wednesday,674.666667


In [13]:
chicago_avg_by_type.head()

,Primary Type,avg_crimes_per_day
0,ARSON,1.855457
1,ASSAULT,61.860835
2,BATTERY,120.948413
3,BURGLARY,22.315476
4,CONCEALED CARRY LICENSE VIOLATION,1.372881


It is very interesting to note that on both datasets Sunday is either the lowest, or close to the lowest crimes per day. Hopefully we will be able to see meaningful changes due to this fact.

In [86]:
#Making a csv of phili 2024/25 crime

philly_2024 = pd.read_csv("data/philadelphia_crime_data_2024.csv")
philly_2025 = pd.read_csv("data/philadelphia_crime_data_2025.csv")

philly_crime_24_25 = pd.concat([philly_2024, philly_2025], ignore_index=True)


In [92]:
#Eagles gameday info
eagles_games_2024 = nfl_game_date(2024, "Eagles").copy()
eagles_games_2024["date"] = pd.to_datetime(eagles_games_2024["Date"]).dt.date
eagles_game_days = set(eagles_games_2024["date"])

#Prepare phili crime date
philly_crime_24_25["dispatch_date_time"] = pd.to_datetime(philly_crime_24_25["dispatch_date_time"])
philly_crime_24_25["date"] = philly_crime_24_25["dispatch_date_time"].dt.date
philly_crime_24_25["is_eagles_game_day"] = philly_crime_24_25["date"].isin(eagles_game_days)

#filter to only the relevant crimes
relevant_philly_crimes = [
    "Aggravated Assault Firearm",
    "Aggravated Assault No Firearm",
    "Robbery Firearm",
    "Robbery No Firearm",
    "Liquor Law Violations",
    "Disorderly Conduct",
    "Public Drunkenness"
]

#filter to these crimes on eagles days
filtered_eagles = philly_crime_24_25[
    (philly_crime_24_25["text_general_code"].isin(relevant_philly_crimes)) &
    (philly_crime_24_25["is_eagles_game_day"])
].copy()

#count number of eagles games to compute average
num_eagles_game_days = len(eagles_game_days)

#table with count of each crime per game day
eagles_crimes_by_day = (
    filtered_eagles.groupby(["text_general_code", "date"])
    .size()
    .reset_index(name="crime_count")
    .pivot(index="text_general_code", columns="date", values="crime_count")
    .fillna(0)
    .astype(int)
    .reset_index()
)

#calculate baseline number of crimes from nomal day
philly_daily_counts = (
    philly_crime_24_25[
        philly_crime_24_25["text_general_code"].isin(relevant_philly_crimes)
    ]
    .groupby(["text_general_code", "date"])
    .size()
    .reset_index(name="daily_count")
)

philly_avg_daily = (
    philly_daily_counts.groupby("text_general_code")["daily_count"]
    .mean()
    .reset_index()
    .rename(columns={"daily_count": "avg_crimes_per_day"})
)

#calc average number of eagles days
avg_crimes_on_game_day = (
    filtered_eagles.groupby("text_general_code")
    .size()
    .div(num_eagles_game_days)
    .reset_index(name="avg_crimes_on_eagles_game_day")
)

#merge
merged = pd.merge(philly_avg_daily, eagles_crimes_by_day, on="text_general_code", how="right")
merged = pd.merge(merged, avg_crimes_on_game_day, on="text_general_code", how="left")

#Add day of the week to each game
eagles_games_2024["DayOfWeek"] = pd.to_datetime(eagles_games_2024["Date"]).dt.day_name()
eagles_games_2024["date"] = pd.to_datetime(eagles_games_2024["Date"]).dt.date

#addinng result, day of week, home/away to table
game_meta = eagles_games_2024.set_index("date")[["Result", "Location", "DayOfWeek"]].T
game_meta.columns.name = None  # Remove column label name for clean display
eagles_crimes_full = pd.concat([merged, game_meta], axis=0)



In [94]:
with pd.option_context('display.max_columns', None):
    display(eagles_crimes_full) 


,text_general_code,avg_crimes_per_day,2024-08-09,2024-08-15,2024-08-24,2024-09-06,2024-09-16,2024-09-22,2024-09-29,2024-10-13,2024-10-20,2024-10-27,2024-11-03,2024-11-10,2024-11-14,2024-11-24,2024-12-01,2024-12-08,2024-12-15,2024-12-22,2024-12-29,2025-01-05,2025-01-12,2025-01-19,2025-01-26,2025-02-09,avg_crimes_on_eagles_game_day
0,Aggravated Assault Firearm,7.257367,7,8,7,3,12,5,7,6,9,11,10,5,6,8,5,10,8,2,7,10,7,5,7,7,7.166667
1,Aggravated Assault No Firearm,14.735294,18,24,23,19,13,17,23,11,13,9,16,11,16,13,9,11,11,13,14,11,18,8,14,18,14.708333
2,Disorderly Conduct,2.301724,0,1,2,2,1,2,0,0,0,4,5,1,1,1,0,0,1,1,0,0,0,4,6,3,1.458333
3,Liquor Law Violations,1.318182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.041667
4,Public Drunkenness,1.234043,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.041667
5,Robbery Firearm,4.352823,2,2,4,4,3,7,5,7,3,1,3,2,5,6,5,8,4,5,3,2,3,2,6,3,3.958333
6,Robbery No Firearm,6.810651,3,7,7,10,10,16,6,5,13,3,5,9,10,3,7,7,3,6,7,5,2,5,6,3,6.583333
Result,NaN,NaN,Win,Win,Loss,Win,Loss,Win,Loss,Win,Win,Win,Win,Win,Win,Win,Win,Win,Win,Loss,Win,Win,Win,Win,Win,Win,NaN
Location,NaN,NaN,Away,Away,Home,Home,Home,Away,Away,Home,Away,Away,Home,Away,Home,Away,Away,Home,Home,Away,Home,Home,Home,Home,Home,Home,NaN
DayOfWeek,NaN,NaN,Friday,Thursday,Saturday,Friday,Monday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Thursday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,Sunday,NaN


In [110]:
# Sixers gameday info
sixers_games_2024 = nba_game_date("76ers", 2024).copy()
sixers_games_2024["date"] = pd.to_datetime(sixers_games_2024["Date"]).dt.date
sixers_game_days = set(sixers_games_2024["date"])

# Prepare phili crime date (already assumed datetime conversion done earlier)
philly_crime_24_25["is_sixers_game_day"] = philly_crime_24_25["date"].isin(sixers_game_days)

# Filter to only the relevant crimes
relevant_philly_crimes = [
    "Aggravated Assault Firearm",
    "Aggravated Assault No Firearm",
    "Robbery Firearm",
    "Robbery No Firearm",
    "Liquor Law Violations",
    "Disorderly Conduct",
    "Public Drunkenness"
]

# Filter to these crimes on Sixers game days
filtered_sixers = philly_crime_24_25[
    (philly_crime_24_25["text_general_code"].isin(relevant_philly_crimes)) &
    (philly_crime_24_25["is_sixers_game_day"])
].copy()

# Count number of Sixers games to compute average
num_sixers_game_days = len(sixers_game_days)

# Table with count of each crime per game day
sixers_crimes_by_day = (
    filtered_sixers.groupby(["text_general_code", "date"])
    .size()
    .reset_index(name="crime_count")
    .pivot(index="text_general_code", columns="date", values="crime_count")
    .fillna(0)
    .astype(int)
    .reset_index()
)

# Calculate baseline number of crimes from normal days
philly_daily_counts = (
    philly_crime_24_25[
        philly_crime_24_25["text_general_code"].isin(relevant_philly_crimes)
    ]
    .groupby(["text_general_code", "date"])
    .size()
    .reset_index(name="daily_count")
)

philly_avg_daily = (
    philly_daily_counts.groupby("text_general_code")["daily_count"]
    .mean()
    .reset_index()
    .rename(columns={"daily_count": "avg_crimes_per_day"})
)

# Calc average number of crimes on Sixers game days
avg_crimes_on_sixers_day = (
    filtered_sixers.groupby("text_general_code")
    .size()
    .div(num_sixers_game_days)
    .reset_index(name="avg_crimes_on_sixers_game_day")
)

# Merge
merged_sixers = pd.merge(philly_avg_daily, sixers_crimes_by_day, on="text_general_code", how="right")
merged_sixers = pd.merge(merged_sixers, avg_crimes_on_sixers_day, on="text_general_code", how="left")

# Add day of the week to each game
sixers_games_2024["DayOfWeek"] = pd.to_datetime(sixers_games_2024["Date"]).dt.day_name()
sixers_games_2024["date"] = pd.to_datetime(sixers_games_2024["Date"]).dt.date

# Adding result, day of week, home/away to table
game_meta_sixers = sixers_games_2024.set_index("date")[["Result", "Location", "DayOfWeek"]].T
game_meta_sixers.columns.name = None  # Remove column label name for clean display
sixers_crimes_full = pd.concat([merged_sixers, game_meta_sixers], axis=0)


In [112]:
with pd.option_context('display.max_columns', None):
    display(sixers_crimes_full)

,text_general_code,avg_crimes_per_day,2024-10-11,2024-10-12,2024-10-14,2024-10-16,2024-10-18,2024-10-23,2024-10-25,2024-10-27,2024-10-30,2024-11-02,2024-11-04,2024-11-06,2024-11-08,2024-11-10,2024-11-12,2024-11-13,2024-11-15,2024-11-18,2024-11-20,2024-11-22,2024-11-24,2024-11-27,2024-11-30,2024-12-03,2024-12-04,2024-12-06,2024-12-08,2024-12-13,2024-12-16,2024-12-20,2024-12-21,2024-12-23,2024-12-25,2024-12-28,2024-12-30,2025-01-01,2025-01-02,2025-01-04,2025-01-06,2025-01-08,2025-01-10,2025-01-12,2025-01-14,2025-01-15,2025-01-18,2025-01-19,2025-01-21,2025-01-24,2025-01-25,2025-01-28,2025-01-29,2025-01-31,2025-02-02,2025-02-04,2025-02-05,2025-02-07,2025-02-09,2025-02-11,2025-02-12,2025-02-20,2025-02-22,2025-02-24,2025-02-26,2025-03-01,2025-03-03,2025-03-04,2025-03-06,2025-03-09,2025-03-12,2025-03-14,2025-03-16,2025-03-17,2025-03-19,2025-03-21,2025-03-23,2025-03-24,2025-03-26,2025-03-29,2025-03-30,2025-04-01,2025-04-03,2025-04-05,2025-04-07,2025-04-09,2025-04-11,2025-04-13,avg_crimes_on_sixers_game_day,2025-03-10
0,Aggravated Assault Firearm,7.257367,5,7,5,6,4,5,7,11,12,2,9,8,11,5,9,8,5,12,2,8,8,6,5,3,6,4,10,6,4,7,6,5,9,6,7,10,7,7,8,2,6,7,7,3,6,5,4,11,6,8,4,3,8,8,4,5,7,7,4,4,5,9,9,6,7,8,5,7,2,7,3,4,5,3,7,2,7,8,8,6,7,6,4,6,6,8,6.195402,NaN
1,Aggravated Assault No Firearm,14.735294,23,14,20,21,19,14,12,9,16,11,27,21,17,11,12,14,10,13,27,14,13,12,13,13,12,13,11,7,10,13,11,16,8,12,17,14,18,17,11,14,11,18,9,9,12,8,13,14,15,13,10,13,10,18,10,11,18,10,13,9,12,9,7,10,16,13,17,9,17,12,7,15,9,11,12,13,7,15,12,15,16,21,10,17,10,12,13.195402,NaN
2,Disorderly Conduct,2.301724,3,1,3,1,2,1,0,4,0,4,3,9,2,1,1,0,1,0,3,0,1,0,1,3,3,2,0,0,2,2,0,2,0,1,0,3,3,1,0,1,0,0,1,1,1,4,0,2,0,1,0,3,0,1,2,1,3,1,6,1,3,0,2,1,1,1,0,0,10,0,3,0,10,4,4,0,12,3,13,3,3,2,3,10,3,0,2.103448,NaN
3,Liquor Law Violations,1.318182,0,1,0,2,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.206897,NaN
4,Public Drunkenness,1.234043,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0.114943,NaN
5,Robbery Firearm,4.352823,7,1,5,5,7,4,2,1,6,5,7,3,2,2,6,4,3,6,10,3,6,6,5,4,8,2,8,8,1,5,5,3,3,4,8,4,1,6,7,2,10,3,7,7,3,2,7,1,3,4,5,3,2,6,1,4,3,2,6,7,2,4,3,1,4,2,1,1,0,3,1,0,2,2,3,1,3,3,2,1,3,5,3,5,3,0,3.781609,NaN
6,Robbery No Firearm,6.810651,9,9,7,8,10,11,7,3,6,11,10,11,10,9,8,7,5,7,9,8,3,8,7,1,3,8,7,7,4,6,3,4,6,3,9,9,6,5,9,5,7,2,5,8,3,5,4,8,9,5,10,0,5,7,10,8,3,8,5,7,4,7,8,8,6,3,10,2,0,8,1,5,5,6,2,3,6,6,4,7,3,7,4,9,3,6,6.091954,NaN
Result,NaN,NaN,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,True,False,True,True,False,True,True,False,True,True,True,True,False,False,True,False,True,False,False,False,False,False,False,False,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,NaN,False
Location,NaN,NaN,Away,Away,Away,Home,Away,Home,Away,Away,Home,Home,Away,Away,Away,Home,Home,Home,Away,Away,Away,Home,Home,Home,Away,Away,Home,Home,Away,Home,Away,Home,Away,Home,Away,Away,Away,Away,Away,Away,Home,Home,Home,Away,Home,Home,Away,Away,Away,Home,Away,Home,Home,Home,Home,Home,Home,Away,Away,Home,Away,Home,Home,Home,Away,Home,Home,Away,Away,Home,Away,Home,Away,Away,Away,Away,Away,Away,Home,Home,Home,Away,Home,Home,Away,Away,Home,Home,NaN,Away
DayOfWeek,NaN,NaN,Friday,Saturday,Monday,Wednesday,Friday,Wednesday,Friday,Sunday,Wednesday,Saturday,Monday,Wednesday,Friday,Sunday,Tuesday,Wednesday,Friday,Monday,Wednesday,Friday,Sunday,Wednesday,Saturday,Tuesday,Wednesday,Friday,Sunday,Friday,Monday,Friday,Saturday,Monday,Wednesday,Saturday,Monday,Wednesday,Thursday,Saturday,Monday,Wednesday,Friday,Sunday,Tuesday,Wednesday,Sat